# Tuần 1-2: Phân tích bài toán, tìm hiểu dataset, xây dựng baseline

In [ ]:
%load_ext autoreload
%autoreload 2

from src.utils import download_data
from src.eda import *
from src.dataloader import load_captions_data
from src.preprocess import *
from src.train import *
from src.evaluate import Evaluator
import os


Tải dữ liệu từ kaggle

[utils.py](../src/utils.py)

In [ ]:
download_data()

## 1. Phân tích dataset

In [ ]:
if os.path.exists(Config.CAPTION):
    data = load_captions_data(Config.CAPTION)
else:
    print(f"LỖI: Không tìm thấy file tại {Config.CAPTION}")
    data = {}

# --- HIỂN THỊ CẤU TRÚC CỦA BIẾN DATA ---
if len(data) > 0:
    first_key = list(data.keys())[0]
    print(f"\nVí dụ cấu trúc biến 'data':")
    print(f"Key (Tên ảnh): {first_key}")
    print(f"Value (List Captions): {data[first_key]}")

1. Phân tích Caption

In [ ]:
caption_stats = analyze_caption_lengths(data)

2. Phân tích Từ vựng

In [ ]:
vocab_counter = analyze_vocabulary(data, top_n=25)

3. Phân tích Ảnh (Chạy trên 1000 ảnh mẫu cho nhanh)

In [ ]:
analyze_image_specs(data, Config.IMG_DIR)

4. Kiểm tra file

In [ ]:
check_dataset_integrity(data, Config.IMG_DIR)

5. Xem ảnh mẫu

In [ ]:
visualize_random_samples(data, Config.IMG_DIR, num_samples=4)

## 2. Tiền xử lý dữ liệu

In [ ]:
train_loader, val_loader, test_loader, vocab = get_loaders(Config, data)

In [ ]:
imgs, captions = next(iter(train_loader))
print(f"Images Shape: {imgs.shape}")
print(f"Captions Shape: {captions.shape}")

In [ ]:
sample_cap = captions[0]
print(f"Tensor: {sample_cap}")
print(f"Decoded: {vocab.denumericalize(sample_cap)}")

## 3. Xây dựng Model: CNN Encoder + LSTM Decoder

### a. CNN Encoder (ResNet-18 Pretrained)
- Sử dụng ResNet-18 pretrained trên ImageNet
- Loại bỏ FC layer cuối, thay bằng một fully connected layer để project feature về embedding size

### b. LSTM Decoder
- Nhận image embedding làm hidden state khởi tạo
- Sử dụng teacher forcing trong quá trình training
- Output: distribution over vocabulary tại mỗi timestep

Kiểm tra kiến trúc model

In [ ]:
# Tạo dữ liệu giả
dummy_img = torch.randn(4, 3, 224, 224).to(Config.DEVICE) # Batch 4 ảnh
dummy_captions = torch.randint(0, 100, (4, 15)).to(Config.DEVICE) # Batch 4 câu, dài 15
# 2. Khởi tạo model
model = CNNtoRNN(
    embed_size=Config.EMBED_SIZE,
    hidden_size=Config.HIDDEN_SIZE,
    vocab_size=3000, # Giả sử vocab có 3000 từ
    num_layers=Config.NUM_LAYERS,
    train_cnn=Config.TRAIN_CNN,
    dropout=Config.DROPOUT
).to(Config.DEVICE)

# 3. Chạy thử Forward Pass
print("Running forward pass...")
outputs = model(dummy_img, dummy_captions)

print(f"Input Image: {dummy_img.shape}")
print(f"Input Caption: {dummy_captions.shape}")
print(f"Output Logits: {outputs.shape}")

# Kiểm tra kích thước output
# Mong đợi: [4, 14, 3000] (14 = 15 - 1 vì bỏ <EOS>)
expected_shape = (4, 14, 3000)

if outputs.shape == expected_shape:
    print("✅ MODEL SHAPE MATCH! Ready for training.")
else:
    print(f"❌ SHAPE MISMATCH! Expected {expected_shape}, got {outputs.shape}")

In [ ]:
run_training(train_loader, val_loader, test_loader, vocab)

## 4. Đánh giá mô hình

In [ ]:


# 1. Load lại model tốt nhất
checkpoint_path = "checkpoints/bs32_lr0.0003_emb256_20260127_2030.pth" # Ví dụ
checkpoint = torch.load(checkpoint_path, map_location=Config.DEVICE)

# 2. Khởi tạo lại kiến trúc model
# Lưu ý: Phải dùng đúng vocab và tham số lúc train
model = CNNtoRNN(
    embed_size=Config.EMBED_SIZE,
    hidden_size=Config.HIDDEN_SIZE,
    vocab_size=len(vocab),
    num_layers=Config.NUM_LAYERS,
    train_cnn=Config.TRAIN_CNN,
    dropout=Config.DROPOUT
).to(Config.DEVICE)

# 3. Load trọng số (Weights)
model.load_state_dict(checkpoint['state_dict'])
print("✅ Model loaded successfully!")

# --- BẮT ĐẦU ĐÁNH GIÁ ---
evaluator = Evaluator(model, vocab, Config.DEVICE)

# Bước 1: Tính điểm số
evaluator.calculate_metrics(test_loader)

# Bước 2: Xem ảnh minh họa (Validation/Test)
evaluator.visualize(test_loader)

# Bước 3: Thử với ảnh ngoài
# evaluator.predict_image("test_image.jpg")